<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Lab-2:-Gridworld" data-toc-modified-id="Lab-2:-Gridworld-1">Lab 2: Gridworld</a></span></li><li><span><a href="#Learning-Outcomes" data-toc-modified-id="Learning-Outcomes-2">Learning Outcomes</a></span></li><li><span><a href="#(Grid)-World-Building" data-toc-modified-id="(Grid)-World-Building-3">(Grid) World Building</a></span></li><li><span><a href="#Markov-decision-process-(MDP)-Class" data-toc-modified-id="Markov-decision-process-(MDP)-Class-4">Markov decision process (MDP) Class</a></span></li><li><span><a href="#Q-value-Function" data-toc-modified-id="Q-value-Function-5">Q-value Function</a></span></li><li><span><a href="#Value-Iteration-Function" data-toc-modified-id="Value-Iteration-Function-6">Value Iteration Function</a></span></li><li><span><a href="#Hints" data-toc-modified-id="Hints-7">Hints</a></span></li></ul></div>

Lab 2: Gridworld
-----

Learning Outcomes
----

__By the end of this session, you should be able to__:

- Implement the Q-Value and Value Iteration functions to calculate the utilities in a grid-world.

(Grid) World Building
-----

In [45]:
reset -fs

In [46]:
"""
The gridworld from lecture as a lists-of-lists.
Each cell has a reward value.
`None` for an obstacle / unreachable state. 
"""

r = -0.04 # The default is a small negative reward to encourage moving towards an optimal terminal state
warehouse= [[r,    r, r, +1],
            [r, None, r, -1],
            [r,    r, r,  r]]

In [47]:
"""
Test code for warehouse data structure.
This cell should NOT give any errors when it is run.
"""

assert len(warehouse) == 3 # Number of rows
assert [len(row) for row in warehouse] == [4, 4, 4] # Number of columns in each row
assert ["Found a None" for row in warehouse if None in row] == ["Found a None"] # Check for a single None
assert [sum(row) for row in warehouse if None not in row] == [0.88, -0.16] # For complete rows
assert [max(row) for row in warehouse if None not in row] == [1, -0.04] # For complete rows
assert [sum(row) for row in list(map(list, zip(*warehouse))) if None not in row] == [-0.12, -0.12, -0.04] # For complete columns
assert [min(row) for row in list(map(list, zip(*warehouse))) if None not in row] == [-0.04, -0.04, -1] # For complete columns

Markov decision process (MDP) Class
------

A Markov decision process (MDP) class for a two-dimensional grid-world has been implemented for you.

Please read carefully through it

In [48]:
class MarkovDecisionProcess():
    "A two-dimensional grid Markov decision process (MDP)."

    def __init__(self, grid, terminals, gamma=1):
        grid.reverse() # Row 0 is on bottom, not on top; The difference between gridworld and Python
        self.grid = grid
        self.terminals = terminals
        self.gamma = gamma
        self.n_rows = len(grid)
        self.n_cols = len(grid[0])

        states = set()
        reward = {}
        for x in range(self.n_cols):
            for y in range(self.n_rows):
                if grid[y][x]:
                    states.add((x, y))
                    reward[(x, y)] = grid[y][x]
                else:
                    reward[(x, y)] = None
        self.reward = reward
        self.states = states
        
        self.actlist = [(1, 0), (0, 1), (-1, 0), (0, -1)] # An action is an (x, y) unit vector; e.g. (1, 0) means move east.
        
        transitions = {}
        for s in states:
            transitions[s] = {}
            for a in self.actlist:
                transitions[s][a] = self.calculate_t(s, a)
        self.transitions=transitions
        
    def actions(self, state):
        """Return a list of actions that can be performed in this state. By default, a
        fixed list of actions, except for terminal states. Override this
        method if you need to specialize by state."""

        if state in self.terminals:
            return [None]
        else:
            return self.actlist

    def calculate_t(self, state, action):
        "Caculate the transition, including probability of not completing intended action."
        if action:
            return [(0.8, self.go(state, action)), # Intended
                    (0.1, self.go(state, self.turn_right(action))), # Error
                    (0.1, self.go(state, self.turn_left(action)))]  # Error
        else:
            return [(0.0, state)]
    
    def t(self, state, action):
        "Probability transition function, aka model of world"
        return self.transitions[state][action] if action else [(0.0, state)]
    
    def turn_heading(self, heading, inc, headings=[(1, 0), (0, 1), (-1, 0), (0, -1)]):
        return headings[(headings.index(heading) + inc) % len(headings)]

    def turn_right(self, heading):
        return self.turn_heading(heading, -1)

    def turn_left(self, heading):
        return self.turn_heading(heading, +1)

    def get_reward(self, state):
        "Return a numeric reward for this state."
        return self.reward[state]
 
    def go(self, state, direction):
        "Return the state that results from going in this direction."
        state_prime = tuple(map(lambda x, y: x+y, state, direction))
        return state_prime if state_prime in self.states else state


In [49]:
# Instantiate warehouse as an instance of MarkovDecisionProcess
warehouse_mdp = MarkovDecisionProcess(grid=warehouse,
                                      terminals=[(3, 2), 
                                                 (3, 1)])

Q-value Function
-----

In [50]:
"""Write q_value function

See lecture notes for pseudocode.
"""

def q_value(mdp, state, action, U):
    "The excepted utility of taking a given action in a given state in a given environment."

    # TODO: Write your solution below here
    


In [51]:
"""
Test code for q_value function.
This cell should NOT give any errors when it is run.
"""

default_params = dict(mdp=warehouse_mdp,
                      state=(0, 0),
                      action=(1, 0),
                      U={s:warehouse_mdp.get_reward(s) for s in warehouse_mdp.states} # Fake values
                      )

assert round(q_value(**default_params), 4) == -0.080

# A state closer to postive terminal
default_params['state']=(2, 2)
assert q_value(**default_params) == 0.752

# Different action in that state
default_params['state']=(2, 2)
default_params['action']=(0, 1)
assert q_value(**default_params) == 0.024

Value Iteration Function
----

In [52]:
"""Write value iteration function to solve mdp.

See lecture notes for pseudocode.
"""

def value_iteration(mdp, epsilon=0.001)->dict:
    "Return the calculate utility of each state with value iteration."
    
    # TODO: Write your solution below here
    


In [53]:
"""
Test code for value_iteration function.
This cell should NOT give any errors when it is run.
"""

mdp_values = value_iteration(warehouse_mdp)
assert type(mdp_values) == dict
assert len(mdp_values.keys()) == 11
assert sorted(mdp_values.keys()) == [\
 (0, 0),
 (0, 1),
 (0, 2),
 (1, 0),
 (1, 2),
 (2, 0),
 (2, 1),
 (2, 2),
 (3, 0),
 (3, 1),
 (3, 2)
]

In [54]:
"""
Test code for value_iteration function.
This cell should NOT give any errors when it is run.
"""
mdp_values = value_iteration(warehouse_mdp)
assert mdp_values[(3, 1)] == -1.0
assert mdp_values[(3, 2)] ==  1.0

In [55]:
"""
Test code for value_iteration function.
This cell should NOT give any errors when it is run.
"""
mdp_values = value_iteration(warehouse_mdp)
try: 
    mdp_values[(1, 1)] # Look for key that should not be there
    assert False
except KeyError: # If key is not there, unit test passes
    assert True

In [56]:
expected_values = {\
 (0, 0): 0.7053082191780823,
 (0, 1): 0.7615582191780823,
 (0, 2): 0.8115582191780822,
 (1, 0): 0.6553082191780822,
 (1, 2): 0.8678082191780823,
 (2, 0): 0.6114155251141552,
 (2, 1): 0.6602739726027398,
 (2, 2): 0.9178082191780822,
 (3, 0): 0.3879249112125825,
 (3, 1): -1.0,
 (3, 2):  1.0}

In [57]:
"""
Test code for value_iteration function.
Approximate values
This cell should NOT give any errors when it is run.
"""

computed_values = dict(sorted(value_iteration(warehouse_mdp, epsilon=1).items())) # Sort solution for quick visual inspection
rounded_computed_values = {k: round(v, 2) for k,v in computed_values.items()}
rounded_expected_values = {k: round(v, 2) for k,v in expected_values.items()}
assert rounded_computed_values == rounded_expected_values

In [58]:
"""
Test code for value_iteration function.
Precisely correctly values
This cell should NOT give any errors when it is run.
"""

computed_values = dict(sorted(value_iteration(warehouse_mdp, epsilon=1).items())) # Sort solution for quick visual inspection
assert rounded_computed_values == rounded_expected_values

Hints
-------

- Pay attention to indexing. Gridworld examples in books and in class use different indexing than Python!
    - Storing in `dict` helps (compared to a list-of-lists).
- Start with direct translation of pseudocode. 
- Get unit tests to pass.
- Then refactor - replace mathematical symbols and concepts with Pythonic names and idioms.
- It might be hard for you. Several of the previous students thought this was the most the difficult lab.  

<br>
<br> 
<br>

----